In [1]:
from tkinter import *
from tkinter import filedialog
from PIL import Image
from PIL import ImageTk
from PIL import ImageTk, Image  
import cv2
import tkinter as tk
import imutils
import numpy as np
# importing os module  
import os
import tkinter.messagebox


In [2]:
def reiniciar_count():
    global count
    count = 0

In [3]:
image_face = None
def elegir_imagen():
    # Especificar los tipos de archivos, para elegir solo a las imágenes
    path_image = filedialog.askopenfilename(filetypes = [
        ("image", ".jpeg"),
        ("image", ".png"),
        ("image", ".jpg")])
    if len(path_image) > 0:
        global image
        #path
        global image_face
        image_face = path_image
        print(image_face)
        # Leer la imagen de entrada y la redimensionamos
        image = cv2.imread(path_image)
        image= imutils.resize(image, height=380)
        # Para visualizar la imagen de entrada en la GUI
        imageToShow= imutils.resize(image, width=180)
        imageToShow = cv2.cvtColor(imageToShow, cv2.COLOR_BGR2RGB)
        im = Image.fromarray(imageToShow )
        img = ImageTk.PhotoImage(image=im)
        lblInputImage.configure(image=img)
        lblInputImage.image = img
        # Label IMAGEN DE ENTRADA
        lblInfo1 = Label(root, text="IMAGEN DE ENTRADA:")
        lblInfo1.place(x=5,y=50)
        # Al momento que leemos la imagen de entrada, vaciamos
        # la iamgen de salida y se limpia la selección de los
        # radiobutton
        lblOutputImage.image = ""



In [29]:
video = None
def video_Stream():
	global video
	video = cv2.VideoCapture(0,cv2.CAP_DSHOW)
	iniciar()

In [5]:
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
count = 0
def iniciar():
    global video
    ret, frame = video.read()
    frame = cv2.flip(frame,1)
    if ret ==True:
        frame = imutils.resize(frame,width=400, height=300)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #cv2.COLOR_BGR2GRAY
        auxFrame = frame.copy()
        auxFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faceClassif.detectMultiScale(frame,1.2,5)
        #print(cap_face)
        if cap_face==True:
            #count = 0
            personName = txtNombre.get()
            #print(personName)
            #print(count)
            dataPath = 'data'
            personPath = dataPath+"/"+personName
            if not os.path.exists(personPath):
                os.makedirs(personPath)
            for (x,y,w,h) in faces:
                global count
                cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
                rostro = auxFrame[y:y+h,x:x+w]
                rostro = cv2.resize(rostro,(150,150),interpolation=cv2.INTER_CUBIC)
                cv2.imwrite(personPath + '/rostro_{}.jpg'.format(count),rostro)
                count = count + 1
                #cv2.imshow('frame',frame)
                #k =  cv2.waitKey(1)
                print(count)
                if count >= 50:
                    tkinter.messagebox.showinfo(title="Alterta",message="Captura Exitosa")
                    reiniciar_count()
                    guardar_face()
                    cap_face_bol()
            #count = count + 1
            
        img = Image.fromarray(frame)
        image = ImageTk.PhotoImage(image=img)
        etip_de_video.configure(image=image)
        etip_de_video.image = image
        etip_de_video.after(10, iniciar)

In [6]:
video2 = None
cap_face = False
def video_Stream_principal():
	global video2
	video2 = cv2.VideoCapture(0,cv2.CAP_DSHOW)
	iniciar_principal()

In [7]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read('modeloLBPHFace.xml')
dataPath = 'data' 
imagePaths = os.listdir(dataPath)
print('imagePaths=',imagePaths)
def iniciar_principal():
    global video2
    ret, frame = video2.read()
    frame = cv2.flip(frame,1)
    if ret ==True:
        frame = imutils.resize(frame,width=400, height=300)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #cv2.COLOR_BGR2GRAY
        auxFrame = frame.copy()
        auxFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faceClassif.detectMultiScale(frame,1.2,5)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
            rostro = auxFrame[y:y+h,x:x+w]
            assertrostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
            result = face_recognizer.predict(rostro)
            cv2.putText(frame,'{}'.format(result),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
            if result[1] < 75:
                cv2.rectangle(frame,(x, y+h), (x+w-50, y+h+30),(0,255,0),-1)
                cv2.rectangle(frame,(x, y), (x+w, y+h),(0,255,0),2)
                print(imagePaths[result[0]])
                recog_face(imagePaths[result[0]])
                cv2.putText(frame, '{}'.format(imagePaths[result[0]]), (x,y+h+25),2,0.8,(255,255,255),1,cv2.LINE_AA)
            else:
                cv2.rectangle(frame,(x, y+h), (x+w-50, y+h+30),(0,0,255),-1)
                cv2.rectangle(frame,(x, y), (x+w, y+h),(0,0,255),2)
                recog_face("desconocido")
                cv2.putText(frame,'Desconocido',(x,y+h+25),2,0.8,(255,255,255),1,cv2.LINE_AA)
        img = Image.fromarray(frame)
        image = ImageTk.PhotoImage(image=img)
        etip_video_principal.configure(image=image)
        etip_video_principal.image = image
        etip_video_principal.after(10, iniciar_principal)

imagePaths= ['Josefina', 'Raquel']


In [8]:
def cap_face_bol():
    global cap_face
    print(cap_face)
    imgGuardar = txtNombre.get()
    if is_empty(imgGuardar):
        tkinter.messagebox.showinfo(title="Alerta",message="Nombre Vacio")
    elif image_face==None or is_empty(image_face):
        tkinter.messagebox.showinfo(title="Alerta",message="Imagen Vacia")
    else:
        if cap_face:
            cap_face=False
        else:
            cap_face=True 

In [9]:
def capturar_rostros():
	personName = txtNombre.get()
	print(personName)
	dataPath = 'data'
	personPath = dataPath+"/"+personName
	if not os.path.exists(personPath):
		os.makedirs(personPath)

In [10]:
def reiniciar_count():
    global count
    count = 0

In [11]:
def guardar_face():
    global image_face
    image_path = image_face
    img = cv2.imread(image_path)
    dataPath = 'faces'
    os.chdir(dataPath)
    filename = txtNombre.get()+".jpg"
    cv2.imwrite(filename, img)

In [12]:
def is_empty(txt):
    txt = set(txt)
    return not bool(txt)

In [13]:
def recog_face(jeta):
    x="faces/"+jeta+".jpg"
    image = cv2.imread(x)
    image= imutils.resize(image, height=380)
    imageToShow= imutils.resize(image, width=247)
    imageToShow = cv2.cvtColor(imageToShow, cv2.COLOR_BGR2RGB)
    im = Image.fromarray(imageToShow )
    img = ImageTk.PhotoImage(image=im)
    InputImageRostro.configure(image=img)
    InputImageRostro.image = img

In [14]:
def entrenamiento():
    global dataPath
    #dataPathEntre = 'data'
    peopleList = os.listdir(dataPath)
    print("lista de personas",peopleList)
    labelsEntre = []
    faceDataEntre = []
    labelEntre  = 0
    for nameDir in peopleList:
        personPath = dataPath+'/'+nameDir
        #print("leyendo imagenes")
        for fileName in os.listdir(personPath):
            labelsEntre.append(labelEntre)
            faceDataEntre.append(cv2.imread(personPath+'/'+fileName,0))
            image = cv2.imread(personPath+'/'+fileName,0)
        labelEntre = labelEntre + 1
    face_recognizer = cv2.face.LBPHFaceRecognizer_create()
    print("entrenando....")

    face_recognizer.train(faceDataEntre, np.array(labelsEntre))

    face_recognizer.write('modeloLBPHFace.xml')

    #face_recognizer.write('modeloFisherFace.xml')
    print("modelo almanecado")
    tkinter.messagebox.showinfo(title="Alerta",message="Registro Exitoso")

In [15]:
##ocultar elemtos del capturador

In [16]:
def registro_vetana():
    registroVentana = Tk()
    registroVentana.title("Reconocimiento Facial")
    registroVentana.geometry("1000x600")
    # Label donde se presentará la imagen de entrada
    lblInputImage = Label(registroVentana)
    lblInputImage.grid(column=0, row=2)
    # Label donde se presentará la imagen de salida
    lblOutputImage = Label(registroVentana)
    lblOutputImage.grid(column=1, row=1, rowspan=6)
    #botones
    #boton = tk.Button(root, text="iniciar video",bg=fondo_boton,relief = "flat",cursor = "hand2",command=video_Stream,width=15, height=2,font=("Calisto MT",12,"bold"))
    #cap_face = False
    #boton.place(x=180,y=400)
    boton = tk.Button(registroVentana, text="iniciar Captura",cursor = "hand2",width=15,command=cap_face_bol, height=2,font=("Calisto MT",12,"bold"))
    boton.place(x=150,y=400)

    #Etiqueta
    etip_de_video = tk.Label(registroVentana,bg="black")
    etip_de_video.place(x=220,y=50)
    
    #boton entrenar
    botonRegistro = tk.Button(registroVentana, text="iniciar Registro",cursor = "hand2",width=15,command=entrenamiento, height=2,font=("Calisto MT",12,"bold"))
    botonRegistro.place(x=400,y=400)
    # Creamos los radio buttons y la ubicación que estos ocuparán

    labelNombre = tk.Label(registroVentana, text = "Nombre")
    labelNombre.grid(column=0, row=6, ipadx=5, pady=5)

    txtNombre = tk.Entry(registroVentana, width=20)
    txtNombre.grid(column=0, row=7, padx=10, pady=5, sticky=tk.N)
    # Creamos los radio buttons y la ubicación que estos ocuparán

    labelNombre = tk.Label(registroVentana, text = "Nombre")
    labelNombre.grid(column=0, row=6, ipadx=5, pady=5)

    txtNombre = tk.Entry(registroVentana, width=20)
    txtNombre.grid(column=0, row=7, padx=10, pady=5, sticky=tk.N)
    
    # Creamos el botón para elegir la imagen de entrada
    btn = Button(registroVentana, text="Elegir imagen", width=25, command=elegir_imagen)
    btn.grid(column=0, row=0, padx=5, pady=5)
    video_Stream()
    registroVentana.mainloop()

In [17]:
def mostrar_registro():
    capturar.pack_forget()
    principal.pack()
    global video2
    video2.release()
    print("ingreso a mostrar")

In [18]:
def denegar_registro():
    principal.pack_forget()
    capturar.pack()
    #video_Stream()
    global video
    video.release()
    print("ingreso a denegar")

In [19]:
def inicio_frame():
    #principal.pack_forget()
    capturar.pack_forget()

In [20]:
def onPrincipal():
    video_Stream_principal()

In [21]:
def offPrincipal():
    global video2
    video2.release()

In [22]:
def onSecundario():
    video_Stream()

In [23]:
def offSecundario():
    global video
    video.release()

In [24]:
# Creamos la ventana principal
root = Tk()
root.title("Reconocimiento Facial")
root.geometry("1000x500")
root.resizable(width=True, height=True)


''

In [25]:
#elemtos menu
menubar = Menu(root)
root.config(menu=menubar)

filemenu = Menu(menubar, tearoff=0)
filemenu.add_command(label="Nuevo Usuario",command=mostrar_registro)
filemenu.add_command(label="Eliminar Usuario",command=denegar_registro)
filemenu.add_separator()
filemenu.add_command(label="Salir", command=root.quit)

editmenu = Menu(menubar, tearoff=0)
editmenu.add_command(label="Editar Usuario")

helpmenu = Menu(menubar, tearoff=0)
helpmenu.add_command(label="Ayuda")
helpmenu.add_separator()
helpmenu.add_command(label="Acerca de...")

menubar.add_cascade(label="Usuarios", menu=filemenu)
menubar.add_cascade(label="Editar", menu=editmenu)
menubar.add_cascade(label="Ayuda", menu=helpmenu)


In [26]:
##frame de vista principal
principal = Frame(root)
principal.pack()
principal.config(bg="lightblue")
principal.config(width=1000,height=500) 
#principal.pack_forget()
InputImageRostro = Label(principal,bg="black")
InputImageRostro.place(x=500,y=50)
#Etiqueta
etip_video_principal = tk.Label(principal,bg="black")
etip_video_principal.place(x=50,y=50)
botonOnCamara1 = tk.Button(principal, text="enceder camara",cursor = "hand2",width=15,command=onPrincipal, height=2,font=("Calisto MT",12,"bold"))
botonOnCamara1.place(x=150,y=400)
botonOffCamara1 = tk.Button(principal, text="apagar camara",cursor = "hand2",width=15,command=offPrincipal, height=2,font=("Calisto MT",12,"bold"))
botonOffCamara1.place(x=350,y=400)

In [27]:
#frame registro
capturar = Frame(root)
capturar.pack()
capturar.config(bg="red")
capturar.config(width=1000,height=500)
#boton capturar
boton = tk.Button(capturar, text="iniciar Captura",cursor = "hand2",width=15,command=cap_face_bol, height=2,font=("Calisto MT",12,"bold"))
boton.place(x=480,y=400)
#boton 
# Creamos el botón para elegir la imagen de entrada
btn = Button(capturar, text="Elegir imagen", width=25, command=elegir_imagen)
btn.place(x=5,y=5)
lblInputImage = Label(capturar)
lblInputImage.place(x=5,y=80)
# Label donde se presentará la imagen de salida
lblOutputImage = Label(capturar)
lblOutputImage.place(x=5,y=400)

botonOnCamara2 = tk.Button(capturar, text="enceder camara",cursor = "hand2",width=15,command=onSecundario, height=2,font=("Calisto MT",12,"bold"))
botonOnCamara2.place(x=100,y=400)
botonOffCamara2 = tk.Button(capturar, text="apagar camara",cursor = "hand2",width=15,command=offSecundario, height=2,font=("Calisto MT",12,"bold"))
botonOffCamara2.place(x=300,y=400)
#Etiqueta
botonEntrenar = tk.Button(capturar, text="registrar rostro",cursor = "hand2",width=15,command=entrenamiento, height=2,font=("Calisto MT",12,"bold"))
botonEntrenar.place(x=660,y=400)

labelNombre = tk.Label(capturar, text = "Nombre")
labelNombre.place(x=5,y=310)
txtNombre = tk.Entry(capturar, width=20)
txtNombre.place(x=5,y=340)
   
etip_de_video = tk.Label(capturar,bg="black")
etip_de_video.place(x=220,y=50)


In [28]:
#hide_captura()
video_Stream_principal()
#inicio_frame()
root.mainloop()


Josefina


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\lyria\appdata\local\programs\python\python37\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "c:\users\lyria\appdata\local\programs\python\python37\lib\tkinter\__init__.py", line 749, in callit
    func(*args)
  File "C:\Users\Lyria\AppData\Local\Temp\ipykernel_9728\2541095227.py", line 27, in iniciar_principal
    recog_face(imagePaths[result[0]])
  File "C:\Users\Lyria\AppData\Local\Temp\ipykernel_9728\2293190862.py", line 4, in recog_face
    image= imutils.resize(image, height=380)
  File "c:\users\lyria\appdata\local\programs\python\python37\lib\site-packages\imutils\convenience.py", line 69, in resize
    (h, w) = image.shape[:2]
AttributeError: 'NoneType' object has no attribute 'shape'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\lyria\appdata\local\programs\python\python37\lib\tkinter\__init__.py", line 1705, in __call__
   

lista de personas ['Josefina', 'Raquel']
entrenando....
modelo almanecado
ingreso a mostrar
Josefina


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\lyria\appdata\local\programs\python\python37\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\Lyria\AppData\Local\Temp\ipykernel_9728\3549984306.py", line 2, in onPrincipal
    video_Stream_principal()
  File "C:\Users\Lyria\AppData\Local\Temp\ipykernel_9728\3754939754.py", line 6, in video_Stream_principal
    iniciar_principal()
  File "C:\Users\Lyria\AppData\Local\Temp\ipykernel_9728\2541095227.py", line 27, in iniciar_principal
    recog_face(imagePaths[result[0]])
  File "C:\Users\Lyria\AppData\Local\Temp\ipykernel_9728\2293190862.py", line 4, in recog_face
    image= imutils.resize(image, height=380)
  File "c:\users\lyria\appdata\local\programs\python\python37\lib\site-packages\imutils\convenience.py", line 69, in resize
    (h, w) = image.shape[:2]
AttributeError: 'NoneType' object has no attribute 'shape'


Josefina


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\lyria\appdata\local\programs\python\python37\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\Lyria\AppData\Local\Temp\ipykernel_9728\3549984306.py", line 2, in onPrincipal
    video_Stream_principal()
  File "C:\Users\Lyria\AppData\Local\Temp\ipykernel_9728\3754939754.py", line 6, in video_Stream_principal
    iniciar_principal()
  File "C:\Users\Lyria\AppData\Local\Temp\ipykernel_9728\2541095227.py", line 27, in iniciar_principal
    recog_face(imagePaths[result[0]])
  File "C:\Users\Lyria\AppData\Local\Temp\ipykernel_9728\2293190862.py", line 4, in recog_face
    image= imutils.resize(image, height=380)
  File "c:\users\lyria\appdata\local\programs\python\python37\lib\site-packages\imutils\convenience.py", line 69, in resize
    (h, w) = image.shape[:2]
AttributeError: 'NoneType' object has no attribute 'shape'
